In [ ]:
import pandas as pd
import numpy as np


In [6]:
import os
os.getcwd()


'c:\\Users\\DEVADHARSHNI\\Customer-churn-project ADP\\Jyp notebook'

In [7]:
users = pd.read_csv("../dataset/users.csv")
watch = pd.read_csv("../dataset/watch_history.csv")
search = pd.read_csv("../dataset/search_logs.csv")
reco = pd.read_csv("../dataset/recommendation_logs.csv")
reviews = pd.read_csv("../dataset/reviews.csv")


In [8]:
users.head()
users.info()
users.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10300 entries, 0 to 10299
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   user_id                  10300 non-null  object 
 1   email                    10300 non-null  object 
 2   first_name               10300 non-null  object 
 3   last_name                10300 non-null  object 
 4   age                      9071 non-null   float64
 5   gender                   9476 non-null   object 
 6   country                  10300 non-null  object 
 7   state_province           10300 non-null  object 
 8   city                     10300 non-null  object 
 9   subscription_plan        10300 non-null  object 
 10  subscription_start_date  10300 non-null  object 
 11  is_active                10300 non-null  bool   
 12  monthly_spend            9283 non-null   float64
 13  primary_device           10300 non-null  object 
 14  household_size        

user_id                       0
email                         0
first_name                    0
last_name                     0
age                        1229
gender                      824
country                       0
state_province                0
city                          0
subscription_plan             0
subscription_start_date       0
is_active                     0
monthly_spend              1017
primary_device                0
household_size             1545
created_at                    0
dtype: int64

In [9]:
# Fill missing numerical values
users['age'].fillna(users['age'].median(), inplace=True)
users['monthly_spend'].fillna(users['monthly_spend'].median(), inplace=True)
users['household_size'].fillna(users['household_size'].median(), inplace=True)

# Fill missing categorical values
users['gender'].fillna('Unknown', inplace=True)

# Convert date columns
users['subscription_start_date'] = pd.to_datetime(users['subscription_start_date'])
users['created_at'] = pd.to_datetime(users['created_at'])

# Remove duplicate users if any
users.drop_duplicates(subset='user_id', inplace=True)


In [10]:
users.isnull().sum()


user_id                    0
email                      0
first_name                 0
last_name                  0
age                        0
gender                     0
country                    0
state_province             0
city                       0
subscription_plan          0
subscription_start_date    0
is_active                  0
monthly_spend              0
primary_device             0
household_size             0
created_at                 0
dtype: int64

In [11]:
users.dtypes


user_id                            object
email                              object
first_name                         object
last_name                          object
age                               float64
gender                             object
country                            object
state_province                     object
city                               object
subscription_plan                  object
subscription_start_date    datetime64[ns]
is_active                            bool
monthly_spend                     float64
primary_device                     object
household_size                    float64
created_at                 datetime64[ns]
dtype: object

In [14]:
watch['watch_duration_minutes'].fillna(0, inplace=True)
watch['progress_percentage'].fillna(0, inplace=True)

# Remove invalid values
watch = watch[watch['watch_duration_minutes'] >= 0]

watch.drop_duplicates(inplace=True)


In [16]:
search['results_returned'].fillna(0, inplace=True)
search['had_typo'].fillna(False, inplace=True)
search['used_filters'].fillna(False, inplace=True)

search.drop_duplicates(inplace=True)


In [17]:
reco['was_clicked'].fillna(0, inplace=True)
reco['recommendation_score'].fillna(reco['recommendation_score'].mean(), inplace=True)

reco.drop_duplicates(inplace=True)


In [18]:
reviews['rating'].fillna(reviews['rating'].mean(), inplace=True)
reviews['sentiment_score'].fillna(0, inplace=True)

reviews.drop_duplicates(inplace=True)


In [19]:
users['subscription_start_date'] = pd.to_datetime(users['subscription_start_date'])
watch['watch_date'] = pd.to_datetime(watch['watch_date'])
search['search_date'] = pd.to_datetime(search['search_date'])
reviews['review_date'] = pd.to_datetime(reviews['review_date'])


In [20]:
users['account_age_days'] = (
    pd.Timestamp.today() - users['subscription_start_date']
).dt.days


In [21]:
watch_agg = watch.groupby('user_id').agg({
    'watch_duration_minutes': 'sum',
    'progress_percentage': 'mean'
}).reset_index()

watch_agg.columns = [
    'user_id',
    'total_watch_time',
    'avg_completion_percentage'
]


In [22]:
search_agg = search.groupby('user_id').agg({
    'search_query': 'count',
    'results_returned': 'mean'
}).reset_index()

search_agg.columns = [
    'user_id',
    'total_searches',
    'avg_search_results'
]


In [23]:
reco_agg = reco.groupby('user_id').agg({
    'was_clicked': 'mean',
    'recommendation_score': 'mean'
}).reset_index()

reco_agg.columns = [
    'user_id',
    'recommendation_ctr',
    'avg_recommendation_score'
]


In [24]:
reviews_agg = reviews.groupby('user_id').agg({
    'rating': 'mean',
    'sentiment_score': 'mean'
}).reset_index()

reviews_agg.columns = [
    'user_id',
    'avg_rating',
    'avg_sentiment_score'
]


In [25]:
final_df = users.merge(watch_agg, on='user_id', how='left')
final_df = final_df.merge(search_agg, on='user_id', how='left')
final_df = final_df.merge(reco_agg, on='user_id', how='left')
final_df = final_df.merge(reviews_agg, on='user_id', how='left')

final_df.fillna(0, inplace=True)


In [26]:
final_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   user_id                    10000 non-null  object        
 1   email                      10000 non-null  object        
 2   first_name                 10000 non-null  object        
 3   last_name                  10000 non-null  object        
 4   age                        10000 non-null  float64       
 5   gender                     10000 non-null  object        
 6   country                    10000 non-null  object        
 7   state_province             10000 non-null  object        
 8   city                       10000 non-null  object        
 9   subscription_plan          10000 non-null  object        
 10  subscription_start_date    10000 non-null  datetime64[ns]
 11  is_active                  10000 non-null  bool          
 12  month

In [27]:
final_df.isnull().sum()


user_id                      0
email                        0
first_name                   0
last_name                    0
age                          0
gender                       0
country                      0
state_province               0
city                         0
subscription_plan            0
subscription_start_date      0
is_active                    0
monthly_spend                0
primary_device               0
household_size               0
created_at                   0
account_age_days             0
total_watch_time             0
avg_completion_percentage    0
total_searches               0
avg_search_results           0
recommendation_ctr           0
avg_recommendation_score     0
avg_rating                   0
avg_sentiment_score          0
dtype: int64

In [ ]:
final_df['is_active'].value_counts()


is_active
True     8519
False    1481
Name: count, dtype: int64

In [30]:
import os

os.makedirs("cleaned_data", exist_ok=True)


In [ ]:
final_df.to_csv(
    "cleaned_data/netflix_churn_cleaned.csv",
    index=False
)



In [32]:
import os
os.listdir("cleaned_data")


['netflix_churn_cleaned.csv']